In [28]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Standard library
import operator
from typing import Iterable, Any, Callable, Generator
# Others
from numpy.random import normal

## accumulate

* `itertools.accumulate(iterable[, func, *, initial=None])`

Make an iterator that returns accumulated sums, or accumulated results of other binary functions (specified via the optional func argument).

If `func` is supplied, it should be a function of two arguments. Elements of the input iterable may be any type that can be accepted as arguments to func. (For example, with the default operation of addition, elements may be any addable type including Decimal or Fraction.)

Usually, **the number of elements output matches the input iterable**. However, if the keyword argument `initial` is provided, the accumulation leads off with the initial value so that the output has one more element than the input iterable.

In [49]:
def accumulate(iterable: Iterable, func=operator.add, *, initial=None) -> Generator:
    # Obtain iterator from iterable
    it = iter(iterable)
    # Initialize total
    total = initial
    # If no initial value is passed by user
    if initial is None:
        try:
            # Set total as the first value returned by the iterator's next method
            total = next(it)
            # Raise StopIteration of iterator is exhausted
        except StopIteration:
            return
    # First yield to start the generator
    yield total
    # Once the function resumes, begin the loop to use the iterator protocol, calling 'it.__next__()'
    for element in it:
        # The left argument is the previously accumulated value and the right is the update value, 
        # This produces the new accumulated value of the current iteration, to which the name 'total' is rebound
        total = func(total, element)
        # Yield the current total to caller
        yield total

### Running statistics

In [53]:
# Data
data = [3, 4, 6, 2, 1, 9, 0, 7, 5, 8]
# Returns a iterator
accumulate(data)

<generator object accumulate at 0x1138a6a50>

In [10]:
# Running product
list(accumulate(data, operator.mul)) 
# Runing maximum
list(accumulate(data, max))
# Running sum
list(accumulate(data))

[3, 12, 72, 144, 144, 1296, 0, 0, 0, 0]

[3, 4, 6, 6, 6, 9, 9, 9, 9, 9]

[3, 7, 13, 15, 16, 25, 25, 32, 37, 45]

### Finance 

* $1000 * 1.05 + (-90) = 960$

* $960 * 1.05 + (-90) = 918$

* $918 * 1.05 + (-90) = 873.9$

* $...$

In [12]:
cashflows = [1000, -90, -90, -90, -90]
# Amortize a 5% loan of 1000 with 4 annual payments of 90
list(accumulate(cashflows, lambda balance, payment: balance * 1.05 + payment))

[1000, 960.0, 918.0, 873.9000000000001, 827.5950000000001]

### Chaotic recurrence relation

* $x_{n+1}=r x_{n}\left(1-x_{n}\right)$

* $0.4$

* $3.8 * 0.4 * (1 - 0.4) = 0.912$

* $3.8 * 0.912 * (1 - 0.912) = 0.305$

* $3.8 * 0.305 * (1 - 0.305) = 0.805$

* $...$

In [26]:
# Chaotic recurrence relation https://en.wikipedia.org/wiki/Logistic_map
# Notice that the update value is not used in the actual function
logistic_map = lambda accumulated, update:  r * accumulated * (1 - accumulated)
# Variables
r = 3.8
x0 = 0.4
# Initialize inputs
inputs = [x0] * 36   
# This works since each iteration remembers the current total value
[format(x, '.2f') for x in accumulate(inputs, logistic_map)]

['0.40',
 '0.91',
 '0.30',
 '0.81',
 '0.60',
 '0.92',
 '0.29',
 '0.79',
 '0.63',
 '0.88',
 '0.39',
 '0.90',
 '0.33',
 '0.84',
 '0.52',
 '0.95',
 '0.18',
 '0.57',
 '0.93',
 '0.25',
 '0.71',
 '0.79',
 '0.63',
 '0.88',
 '0.39',
 '0.91',
 '0.32',
 '0.83',
 '0.54',
 '0.95',
 '0.20',
 '0.60',
 '0.91',
 '0.30',
 '0.80',
 '0.60']

## reduce

* `functools.reduce(function, iterable[, initializer])`

Apply function of two arguments cumulatively to the items of iterable, from left to right, so as to **reduce the iterable to a single value**. For example, `reduce(lambda x, y: x + y, [1, 2, 3, 4, 5])` calculates `((((1 + 2) + 3) + 4) + 5)`. The left argument, `x`, is the accumulated value and the right argument, `y`, is the update value from the iterable. If the optional `initializer` is present, it is placed before the items of the iterable in the calculation, and serves as a default when the iterable is empty. If `initializer` is not given and iterable contains only one item, the first item is returned.

In [56]:
def reduce(function, iterable: Iterable, initializer=None) -> Any:
    # Obtain iterator from iterable
    it = iter(iterable)
    # If no initializer passed by user
    if initializer is None:
        # Set value as the first value returned by the iterator's next method
        value = next(it)
    else:
        # Bind 'value' to the same object that 'initializer' references
        value = initializer
    # The iterator's 'next' method will be called in the loop to obtain the update value 'element' at each step
    for element in it:
        # The left argument is the previously accumulated value and the right is the update value, 
        # This produces the new accumulated value of the current iteration, to which the name 'value' is rebound
        value = function(value, element)
    return value

## chain

* `itertools.chain(*iterables)`

Make an iterator that returns elements from the first iterable until it is exhausted, then proceeds to the next iterable, until all of the iterables are exhausted. Used for treating consecutive sequences as a single sequence. Notice that this function could only chain together iterables--- not *flatten* nested iterables.

In [57]:
# The *iterables form collects any extra unmatched positional arguments in a tuple
# This essentially creates a tuple with an arbitrary number of iterables as elements--- Tuple[Iterable, ...]--- in the function body
def chain(*iterables: Iterable) -> Generator:
    # For loop calls the tuple object's iter method
    # Each element of the 'iterables' tuple is an interable in and of itself
    for it in iterables:
        # For each iterable in the 'iterables' tuple, the for loop calls that iterable's iter method 
        for element in it:
            # Yield the value of the element to the caller
            yield element

### Examples

In [36]:
chain('ABC', 'DEF')

<generator object chain at 0x11314e820>

In [41]:
tuple(chain('ABC', 'DEF'))
tuple(chain([1, 2, 3, 4], (2, 3, 4, 5)))
list(chain([3, 4], 'abd', {2, 3, 4}))

('A', 'B', 'C', 'D', 'E', 'F')

(1, 2, 3, 4, 2, 3, 4, 5)

[3, 4, 'a', 'b', 'd', 2, 3, 4]

An alternate constructor for `chain()` is to get the chained inputs from a single iterable argument that is evaluated lazily (rather than an arbitrary number of iterables).

* `chain.from_iterable(iterable)`

In [66]:
# No unpacking into a tuple like *args, since it is a single iterable containing other iterables
def from_iterable(iterables: Iterable[Iterable]) -> Generator:
    # For loop calls the iterables' iter to step through the elements of 'iterables'
    for it in iterables:
        for element in it:
            yield element

### Examples

In [59]:
from_iterable(['ABC', 'DEF'])

<generator object from_iterable at 0x11379bac0>

In [64]:
list(from_iterable(['ABC', 'DEF']))
list(from_iterable(['dfd', [2, 2, 4], ['abd']]))

['A', 'B', 'C', 'D', 'E', 'F']

['d', 'f', 'd', 2, 2, 4, 'abd']

## combination

* `itertools.combinations(iterable, r)`

This function returns `r` length subsequences of elements from the input iterable.

The combination tuples are emitted in lexicographic ordering according to the order of the input iterable. So, if the input iterable is sorted, the combination tuples will be produced in sorted order.

Elements are treated as unique based on their position, not on their value. So if the input elements are unique, there will be no repeat values in each combination. On the other hand, if any value is repeated, the function will print similar combinations thinking each value is different.

The implementation uses `for/else` loop:

```
for item in iterable:
    if search_something(item):
        # Found it!
        process(item)
        break
else:
    # Didn't find anything..
    not_found_in_container()
```

The `else` keyword in a `for` loop specifies a block of code to be executed when the loop is finished. The first scenario is when the `break` is encountered. The second scenario is that the loop ends without encountering a `break` statement, in which case the code in the `else` block will be executed.

In [118]:
def combinations(iterable: Iterable, r: int) -> Generator:
    # Pool of values
    pool = tuple(iterable)
    # Number of values in the pool
    n = len(pool)
    
    # There are no ways of choosing 'r' objects from 'n' objects if r > n
    if r > n:
        return
    
    # Initialize indices [0, 1, 2, ..., (r - 2), (r - 1)] in the normal order
    indices = list(range(r))
    # First yield returns the first subsequence (combo) of size r 
    # This yields a tuple (pool[0], pool[1], ..., pool[r-1])
    yield tuple(pool[i] for i in indices)
    
    while True:
        for i in reversed(range(r)):
            if indices[i] != i + n - r:
                break
        else:
            # Run when loop finishes without break
            return
        indices[i] += 1
        for j in range(i+1, r):
            indices[j] = indices[j-1] + 1
        yield tuple(pool[i] for i in indices)

### Control flow

---

* `pool = (1, 3, 5)`

* `n = 3`

* `r = 2`

* `indices = [0, 1]`

* The first yielded combination is `(1, 3)`

---

```
 # Line 1    for i in reversed(range(r)):
 # Line 2         if indices[i] != i + n - r:
 # Line 3             break
 # Line 4    else:
 # Line 5        return
 # Line 6     indices[i] += 1
 # Line 7     for j in range(i+1, r):
 # Line 8         indices[j] = indices[j-1] + 1
 # Line 9    yield tuple(pool[i] for i in indices)
```

---

1. In the first run, `indices = [0, 1]`:

    - Line 1: `for i in reversed(range(2))`  is `for i in (1, 0)`
    - Line 2: `i = 1`, so `indices[1] != 1 + 3 - 2` becomes `1 != 2`, which is `True`
    - Line 3: `for` loop terminates with `break`
    - Line 4: `else` block is not run
    - Line 5: `else` block is not run
    - Line 6: `for` does not localize variables, so `i = 1` and `indices[1] += 1` increments 1 by 1
        - Updated `indices = [0, 2]`
    - Line 7: `for j in range(1 + 1, 2)` is `range(2, 2) = []`, so `StopIteration` is raised and the loop finishes
    - Line 8: `tuple(pool[i] for i in [0, 2])` yields `(1, 5)`

The second combination is `(1, 5)`.

---

2. In the second run when the function picks back up, `indices = [0, 2]`:

    - Line 1: `for i in reversed(range(2))`  is `for i in (1, 0)`
    - Line 2: `i = 0`, so `indices[0] != 0 + 3 - 2` becomes `0 != 1`, which is `True`
    - Line 3: `for` loop terminates with `break`
    - Line 4: `else` block is not run
    - Line 5: `else` block is not run
    - Line 6: `for` does not localize variables, so `i = 0` and `indices[0] += 1` increments 0 by 1
        - Updated `indices = [1, 2]`
    - Line 7: `for j in range(0 + 1, 2)` is `range(1, 2) = [1]`, in which case `j = 1` and `indices[1] = indices[1 - 1] + 1` becomes `indices[1] = 1 + 1 = 2`
        - Updated `indices = [1, 2]`
    - Line 8: `tuple(pool[i] for i in [1, 2])` yields `(3, 5)`

The second combination is `(3, 5)`.

---

1. In the third run when the function picks back up, `indices = [1, 2]`:

    - Line 1: `for i in (1, 0)` exhausted the iterator, so a `StopIteration` is raised and the loop completes
    - Line 4: `else` block is run since the loop finishes without `break`
    - Line 5: `return` finishes the function and returns control to caller




In [249]:
list(combinations((1, 3, 5), 2))

[(1, 3), (1, 5), (3, 5)]

To understand the `while` loop implementation, recall that:

---

* `pool = (1, 3, 5)`

* `n = 3`

* `r = 2`

* `indices = [0, 1]`

* The first yielded combination is `(1, 3)`

---

1. We need to understand that the job of the `while` loop is to increment the `indices` one after the other.

2. The job of the `for i in reversed(range(r))` loop, where `list(reversed(range(r))) = [1, 0]`, is to ensure that we do not over-increment any of values in `[0, 1]`. What the loop precisely makes sure is that:
  
    - The value of `indices[i] = indices[r - 1]` cannot be greater than `n - 1`. This is because `r` (the number of objects to choose) cannot be greater than `n` (the largest possible number of objects to choose from). In our case, the largest index `1` in `[0, 1]` cannot exceed the largest possible index `2` in `0, 1, 2` where `n = 3`. In the general case, we obtain this largest possible index (the upper limit) using `n - 1` since python counts from 0.

    - The value of `indices[i] = indices[r - 2]` cannot be greater than `n - 2`. In our case, the second largest index `0` in `[0, 1]` cannot exceed the second largest possible index `1` in `0, 1, 2`. In the general case, we obtain this second largest possible index using `n - 2`. The reasoning is the same: if we are to choose `r = 1` objects from among `n = 2` possible objects, the value `indices[r - 2] = [0, 1][0] = 0` cannot be greater than the largest index in `0, 1`, which is `1`.

    - In the general case, we have `indices[r - k] < n - k`.

    - Put another way, `n - 1` is upper limit (not inclusive) for incrementing the index `indices[r - 1]`; in our case, `2 (n - 1) > 1 (r - 1)` so we can increment `1` by 1 to get `2`. Then, `n - 2` is the upper limit (not inclusive) for incrementing the second index `indices[r - 2]`; again, in our case, `1 (n - 2) > 0 (r - 2)`, so we can increment `0` by 1 to get `1`. This same patterns continues.

3. The looping variable `i` goes from `r - 1, r - 2, ..., r - k, ..., 2, 1,  0`, or `[1, 0]` in this case, we can substitute the `r - k` with `i` in the general equation above.

\begin{align}
r - k &= i \\
- k &= i - r \\
k & = -i + r \\ 
k & = r - i
\end{align}

1. The general case then becomes `indices[i] < n - (r - i)` $\rightarrow$ `indices[i] < n - r + i`. The official python example uses `indices[i] != i + n - r`, which accomplishes the same thing. When `indices[i] == i + n - r` is `True`, it effectively covers all the cases where the limit `indices[i] < n - r + i` is violated.

2. If `indices[i] != i + n - r` is `True`, it effectively means that we can safely increment `indices[i]` by 1.

3. The loop `for j in range(i+1, r): indices[j] = indices[j-1] + 1` ensures that every time we increment an `indices[i]`, we set the indexes after `indices[i]` to their smallest possible values. This prevents us from missing out on any combination. For instance, suppose we have `(1, 3, 5, 7, 9)` as the iterable, `r = 3`, and the combination indices `[0, 2, 4]`. When we increment 2 to 3, the last index will be 4, in which case we would miss out on the `[0, 2, 3]` combination. But if we update the latter indices based on that the previous by incrementing 2 by 1 and assigning that value as the last index of `[0, 2, 4]` $\rightarrow$ `[0, 2, 3]`, we effectively address the problem.

4. Lastly, `tuple(pool[i] for i in indices)` simply yields the combination of each specific iteration of the `while` loop.

## compress

* `itertools.compress(data, selectors)`

This function makes an iterator that filters elements from `data`, returning only those that have a corresponding element in `selectors` that evaluates to True. Execution stops when either of the `data` or `selectors` iterables has been exhausted.



In [9]:
def compress(data: Iterable, selectors: Iterable) -> Generator:
    # Each 'd' from 'data' is returned if 's' in 'selector' is True or its truth value is True
    # The truth value of 1 is true
    return (d for d, s in zip(data, selectors) if s)

In [3]:
# Generator
compress('ABCDEF', [1,0,1,0,1,1])

<generator object compress.<locals>.<genexpr> at 0x10ca30ac0>

In [12]:
# List of integers
list(compress('ABCDEF', [1, 0, 1, 0, 1, 1]))
# List of booleans
# Execution stops when the shortest iterable exhausts
list(compress('ABCDEF', [True, False, False, True]))
# The truth values of strings are True
list(compress('ABCDEF', ['string', 'true', 'false']))

['A', 'C', 'E', 'F']

['A', 'D']

['A', 'B', 'C']

## count

* `itertools.count(start=0, step=1)`

This generator function makes an iterator that returns evenly spaced values starting with the number `start`. This is often used as an argument to `map(func, *iterables)` to generate consecutive data points. Also, this could be used with `zip()` to add sequence numbers.

In [13]:
def count(start: int=0, step: int=1) -> Generator:
    # Initialize first element
    n = start
    while True:
        # First yield returns the first item
        yield n
        # When the function resumes, add 'step' to 'n' and yield the next item  
        n += step

In [18]:
# Generator 
count(10)
for item in count(10):
    if item <= 30:
        print(item, end=' ')
    else:
        break


<generator object count at 0x10d5db430>

10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 

In [27]:
list = ['John', 'Marie', 'Jack', 'Anna']
# Zip finishes when the shorter iterable 'list' is exhausted
# This emulates the enumerate function 
for i in zip(count(), list):
    print(i)
# Enumerate
for index, name in enumerate(list):
    index, name

(0, 'John')
(1, 'Marie')
(2, 'Jack')
(3, 'Anna')


(0, 'John')

(1, 'Marie')

(2, 'Jack')

(3, 'Anna')